In [40]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

from PIL import Image
from io import BytesIO
import re

import dateparser as dp
import datetime

commons_site = pb.Site("commons", "commons")

In [41]:
from modules.wmtools import is_commons_file, get_hash

In [42]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [ ]:
req_proxy = RequestProxy()

In [78]:
url = 'http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/presidente/Paginas/2015/rajoy-es-entrevistado-en-.aspx'
excluded = []
categories = ['Ana Blanco', 
              'Mariano Rajoy in 2015',
              'October 2015 in Madrid',
              'Televisión Española']

In [79]:
while True:
    request = req_proxy.generate_proxied_request(url)
    if request is not None:
        #print("{0}".format(u''.join(request.text).encode('utf-8')))
        response = request.text
        break
    else :
        print ('Trying')

soup = BeautifulSoup(response, 'html.parser')

2018-01-20 22:43:28,542 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (X11; U; Linux x86_64; pl-PL; rv:2.0) Gecko/20110307 Firefox/4.'}
2018-01-20 22:43:28,545 root   DEBUG    Using proxy: 91.243.242.5:8080 | FreeProxy
2018-01-20 22:43:58,584 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 91.243.242.5:8080 | FreeProxy PL Size = 2177
2018-01-20 22:43:58,589 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (X11; U; Linux x86_64; es-ES; rv:1.9.1.8) Gecko/20100216 Fedora/3.5.8-1.fc11 Firefox/3.5.'}
2018-01-20 22:43:58,591 root   DEBUG    Using proxy: 116.58.243.190:8080 | FreeProxy


Trying


2018-01-20 22:44:28,628 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 116.58.243.190:8080 | FreeProxy PL Size = 2176
2018-01-20 22:44:28,634 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (X11; U; Linux x86_64; rv:1.9.1.1) Gecko/20090716 Linux Firefox/3.5.'}
2018-01-20 22:44:28,637 root   DEBUG    Using proxy: 188.165.251.222:1080 | FreeProxy
2018-01-20 22:44:28,725 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 188.165.251.222:1080 | FreeProxy PL Size = 2175
2018-01-20 22:44:28,731 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_4_11; de-de) AppleWebKit/533.16 (KHTML, like Gecko) Version/4.1 Safari/533.1'}
2018-01-20 22:44:28,733 root   DEBUG    Using proxy: 109.173.9.44:8080 | FreeProxy


Trying
Trying
RR Status 200


In [80]:
title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
#title = u'Segundo Gobierno de Rodríguez Zapatero (IX Legislatura) 2010'
title

'Rajoy es entrevistado en los informativos de TVE'

In [81]:
try:
    date_string = soup.find_all("div", class_="contenidoFecha")[0].get_text()
    pub_date_sp = ' '.join(date_string.strip().split(' ')[-4:])
    parsed_pub_date = dp.parse(pub_date_sp)
    pub_date = parsed_pub_date.strftime('%Y-%m-%d')
    
    year_date = parsed_pub_date.strftime('%Y')
    if 'La Moncloa' in date_string:
        categories.append('Palacio de la Moncloa')
        categories.append(year_date + ' in Madrid')
except :
    pub_date = None

pub_date

'2015-10-20'

In [82]:
p = re.compile("gFIO\[\d{1,2}\] = \\'(.*)\\';")
image_list = ['http://www.lamoncloa.gob.es' + url for url in re.findall(p, response)]
image_list

['http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/261015TVE4.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/261015TVE3.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/261015TVE2.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/261015TVE1.jpg']

In [83]:
p = re.compile("gFT\[\d{1,2}\] = \\'(.*)\\';")
description_list = [desc.strip().rstrip('.') for desc in re.findall(p, response)]
description_list

['El presidente del Gobierno, Mariano Rajoy, es entrevistado por Ana Blanco en los informativos de TVE',
 'El presidente del Gobierno, Mariano Rajoy, es entrevistado por Ana Blanco en los informativos de TVE.<br>',
 'El presidente del Gobierno, Mariano Rajoy, es entrevistado por Ana Blanco en los informativos de TVE.<br>',
 'El presidente del Gobierno, Mariano Rajoy, es entrevistado por Ana Blanco en los informativos de TVE']

In [84]:
p = re.compile("gFA\[\d{1,2}\] = \\'(.*)\\';")
author_list = [author.strip() for author in re.findall(p, response)]
author_list

['Pool Moncloa/Diego Crespo',
 'Pool Moncloa/Diego Crespo',
 'Pool Moncloa/Diego Crespo',
 'Pool Moncloa/Diego Crespo']

In [85]:
truth_list = ['Pool' in author for author in author_list]
truth_list

[True, True, True, True]

In [86]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Galería fotográfica: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

vars = {
    "url": url,
    "description": None,
    "author": None,
    "date": pub_date,
    "title": title,
    "cat_string": '\n'.join(['[[Category:'+i+']]' for i in categories])
}

In [ ]:
used_names = []

global_counter = 1
for i, image in enumerate(image_list):
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
    elif truth_list[i] == False:
        print ("Image from a party other than Moncloa. Skipping")
        continue        
        
    # First, the image is downloaded and stored
    image_url = quote(image.encode('utf-8'), ':/')
    image_name = '{} {:02d}.jpg'.format(title.replace(':', ' -').replace('  ', ' '), global_counter)
    global_counter += 1
    image_path = os.path.join(images_directory, image_name)
    while True:
        print ('Trying download')
        img_request = req_proxy.generate_proxied_request(image_url)
        if img_request is not None:
            image = Image.open(BytesIO(img_request.content))
            image.save(image_path)
            print ('Image downloaded. Starting upload process')
            break
        else :
            print ('Still trying download')

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    vars["description"] = description_list[i]
    vars["author"] = author_list[i]
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot(image_path,
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)

2018-01-20 22:50:07,618 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b11pre) Gecko/20110128 Firefox/4.0b11pr'}
2018-01-20 22:50:07,620 root   DEBUG    Using proxy: 103.74.246.94:42419 | FreeProxy


Trying download
